In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#!pip install import_ipynb
import import_ipynb

In [17]:
! ls "/content/gdrive/My Drive/ColabNotebooks/"

Burnout  CodeForces  Fun  HHModel  Neuromatch  PC  PC_param.ipynb


In [21]:
from google.colab import drive
drive.mount('/content/gdrive')
import sys

path_to_module = '/content/gdrive/MyDrive/ColabNotebooks/PC_param.ipynb'
sys.path.append(path_to_module)


from gdrive.MyDrive.ColabNotebooks.PC_param import default_parameters_network

pars = default_parameters_network()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [23]:

# //////////////  FUNCTIONS ////////////////////////////////////////

# 1- Input/Output function
def phi(I_tot, a, b, hm):
    return hm * (1 / (1 + np.exp(- (a * I_tot + b))))

def phiwongE(I_tot, c = 310, g=.16, Ie= 125):
    return (c * I_tot - Ie) / 1 - np.exp(-g * (c *I_tot - Ie))
def phiwongI(I_tot, c = 615, g =.16, Ii = 125):
    return (c * I_tot - Ii) / 1 - np.exp(-g * (c *I_tot - Ii))



# 2- Input/Output function
def plot_io(x, y, sign):
    if sign == "+":
        sign_name = 'Excitatory'
        label = "ae={0}, be={1}, hme={2}"
        a, b, hm = pars['ae'], pars['be'], pars['hme']
        color = "k"
    elif sign == "-":
        sign_name = 'Inhibitory'
        label = "ai={0}, bi={1}, hmi={2}"
        a, b, hm = pars['ai'], pars['bi'], pars['hmi']
        color = "r"

    plt.plot(x, y, color, label=label.format(a, b, hm))

    plt.xlabel("Input values - nA")
    plt.ylabel("Spike Frequency - Hz")
    plt.xlim([-0.01, 1])
    plt.title("Input-output function")
    # plt.title("{0} Input-output function".format(sign_name))
    plt.legend()


def plot_r(range_sim, r_e, r_i, param, xlim_ar = None):
    label_e = "Excitatoty  Jee={0}, Jei={1}"#, I1={2}"
    label_i = "Inhibitory  Jii={0}, Jie={1}"#, I2={2}"
    plt.plot(range_sim, r_e, "r", label=label_e.format(param.Jee, param.Jei))#, param.I1 #, param.In
    plt.plot(range_sim, r_i, "orange", label=label_i.format(param.Jii, param.Jie)) #, round(param.I2, 2)))

    plt.xlabel("Time - ms")
    plt.ylabel("Spike Frequency - Hz")
    if xlim_ar != None:
      plt.xlim(xlim_ar) #[0, .1]
    plt.title("Firing rate of the NMDA and GABA populations")
    plt.legend()

def plot_s(range_sim, S_e, S_i, param, xlim_ar = None):
    label_e = "Excitatoty  Jee={0}, Jei={1}"#, I1={2}"
    label_i = "Inhibitory  Jii={0}, Jie={1}"#, I2={2}"
    plt.plot(range_sim, S_e, "olive", label=label_e.format(param.Jee, param.Jei)) #, param.I1
    plt.plot(range_sim, S_i, "green", label=label_i.format(param.Jii, param.Jie)) #, round(param.I2, 2)
    if xlim_ar != None:
      plt.xlim(xlim_ar)
    plt.xlabel("Time - ms")
    plt.ylabel("Open channel")
    #plt.xlim([0, .1])
    plt.title("Average open channel for the NMDA and GABA populations")
    plt.legend()

# 2- Plot HeatMap of firing rate function
def HeatMap(rE, rI, J=None):
    if J == None:
        J = [.00989, 0.0081, .1, .87, .00081]  # J = dict(Jin=.008, Jee= .2, Jie=.2, Jei=1.4, Jii=6.7)

    rE_df = pd.DataFrame(rE.T)  # to get time vs pop
    rI_df = pd.DataFrame(rI.T)

    rE_df.index.name, rI_df.index.name = ["Excitatory Population", "Inhibitory Population"]
    rE_df.columns.name, rI_df.columns.name = ["Time s", "Time s"]
    # print(rE_df.loc[[10]])

    # set context for the upcoming plot
    sns.set_context("notebook", font_scale=.8, rc={"lines.linewidth": 2.5, 'font.family': 'Helvetica'})

    fig, (axA, axB) = plt.subplots(2, 1, figsize=(6, 6))

    sns.heatmap(rE_df, ax=axA, cmap="viridis")
    sns.heatmap(rI_df, ax=axB)
    axA.set_title(f"Firing rate in Hz of exc populations over time. Jie: {J[2]}, Jee: {J[1]}, Jin: {J[0]}",
                  fontdict={"fontsize": 10})
    axB.set_title(f"Firing rate in Hz of inh populations over time. Jei: {J[3]}, Jii: {J[4]}",
                  fontdict={"fontsize": 10})
    plt.tight_layout()

    plt.show()


# 3- Compute derivative

def wilsonCowanWong(p1: classmethod, p2: classmethod, sim: classmethod, param: classmethod, k: object, stim: classmethod) -> object:
    dt = sim.dt
    s_ampa, r_e, i_tot_e, dr_e_dt, phi_arr_e = p1.S, p1.r, p1.Itot, p1.drdt, p1.Phi
    s_gaba, r_i, i_tot_i, dr_i_dt, phi_arr_i = p2.S, p2.r, p2.Itot, p2.drdt, p2.Phi
    In = stim.In #, range_sim, w, sigmaIn =stim.sensoryInput(param, sim)
    #I_noise_E = np.random.normal(0, .0007, 1)
    #I_noise_I = np.random.normal(0, .0007, 1)

    i_tot_e[k, :] = param.Jee * s_ampa[k - 1, :] - param.Jie * np.dot(s_gaba[k - 1, :], param.wie) + param.Jin * In[k-1,:]
    i_tot_i[k, :] = param.Jei * np.dot(s_ampa[k - 1, :], param.wei) - param.Jii * np.dot(s_gaba[k - 1, :], param.wii)

    phi_arr_e = phi(i_tot_e[k, :], param.ae, param.be, param.hme) #phiwongE(i_tot_e[k])#
    phi_arr_i = phi(i_tot_i[k, :], param.ai, param.bi, param.hmi) #phiwongI(i_tot_i[k])#

    dr_e_dt[k, :] = (-r_e[k - 1, :] + phi_arr_e) / param.taue
    dr_i_dt[k, :] = (-r_i[k - 1, :] + phi_arr_i) / param.taui

    r_e[k, :] = r_e[k - 1, :] + dr_e_dt[k, :] * dt
    r_i[k, :] = r_i[k - 1, :] + dr_i_dt[k, :] * dt

    dS_amp_dt = (- s_ampa[k - 1, :] / param.tauAMPA) + r_e[k, :]
    s_ampa[k, :] = s_ampa[k - 1, :] + dS_amp_dt * dt


    dS_gab_dt = (- s_gaba[k - 1, :] / param.tauGABA) + r_i[k, :]
    s_gaba[k, :] = s_gaba[k - 1, :] + dS_gab_dt * dt

    return r_e, r_i, s_ampa, s_gaba, i_tot_e, i_tot_i, In



# Filters gauss and Dog

def gaussian_filter(s, N):
    k = np.arange(1, N+1)
    n = 1 / (np.sqrt(2 * np.pi) * N * s)
    gaussW = n * np.exp(-(k - k[:, np.newaxis])**2 / (2 * s**2))
    gaussW = gaussW / np.sum(gaussW)#
    #gaussW = gaussW / (0.01**2 / np.max(gaussW))
    return gaussW

def dog_filter(sIn, sOut, N):
    k = np.arange(1, N+1)
    gaussIn =  np.exp(-(k - k[:, np.newaxis])**2 / (2 * sIn**2))
    gaussOut = np.exp(-(k - k[:, np.newaxis])**2 / (2 * sOut**2))
    dog = gaussOut - gaussIn
    #dog = dog / np.sum(dog)#
    dog = dog/ (0.88**2 / np.max(dog))
    return dog
